- Objective: carbon control by optimization algorithm
- Ideal Carbon mass = Required Oxygen mass * (12/16)
- Required Oxygen mass = m_FeO * (16/71.844) + m_Fe2O3 * (48/159.69) + m_Fe3O4 * (64/231.533) + m_ZnO * (16/65.38)

- Based on the ideal carbon mass, determine the lower/upper boundary for the optimization:
 * Lower boundary = 0.8 * ideal
 * Upper boundary = 1.0 * ideal

In [1]:
import pandas as pd
import numpy as np
from pycaret.regression import *
import optuna

In [2]:
# raw_df = pd.read_csv("Op_feed_composition.csv", encoding='cp949')
raw_df = pd.read_csv("Op_feed_composition_detail.csv", encoding='cp949')
Model_Z = load_model('Tuend_XGB_Z')

Transformation Pipeline and Model Successfully Loaded
[21:56:10] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:1040: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[21:56:10] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:749: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
[21:56:10] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:438: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XG

In [3]:
n = raw_df.shape[0]
result_df = raw_df.copy()
result_df['C_suggest']=None


a_fixed = raw_df.iloc[:,0]/100
b_fixed = raw_df.iloc[:,1]/100
c_fixed = raw_df.iloc[:,2]/100
d_fixed = raw_df.iloc[:,3]/100
e_fixed = raw_df.iloc[:,5]/100

ideal_oxygen = c_fixed * (16/71.844) + d_fixed * (48/159.69) + e_fixed * (81.38/65.38) * (16/81.38)
ideal_carbon = ideal_oxygen * (12/16)

C_lower = ideal_carbon * 0.7
C_upper = ideal_carbon * 1.3

In [5]:
def carbon_callback(trial):
    
    f = trial.suggest_float('C' , C_lower[i], C_upper[i])
 
    test_df = pd.DataFrame({'TFe'   : [a_fixed[i]],
                            'MFe'   : [b_fixed[i]],
                            'FeO'   : [c_fixed[i]],
                            'Fe2O3' : [d_fixed[i]],
                            'C'     : [f],
                            'Zn'    : [e_fixed[i]]
                            })

    prediction_Z = predict_model(Model_Z, data=test_df)
    obj = (prediction_Z['Label'] - 0.82)**2

    

    return(obj)

In [6]:
for i in range(n):
    globals()['study_{}'.format(i)] = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner())
    globals()['study_{}'.format(i)].optimize(carbon_callback, n_trials=600)
    result_df['C_suggest'][i] = list(globals()['study_{}'.format(i)].best_trial.params.values())[0]

[I 2022-10-04 21:56:39,062] A new study created in memory with name: no-name-7a19e807-745c-44cc-a02e-3fa867d5739a
[I 2022-10-04 21:56:39,272] Trial 0 finished with value: 0.004732140339910984 and parameters: {'C': 0.10525208865125843}. Best is trial 0 with value: 0.004732140339910984.
[I 2022-10-04 21:56:39,441] Trial 1 finished with value: 0.007809977512806654 and parameters: {'C': 0.09944091636357887}. Best is trial 0 with value: 0.004732140339910984.
[I 2022-10-04 21:56:39,614] Trial 2 finished with value: 0.005583848338574171 and parameters: {'C': 0.10282079892576268}. Best is trial 0 with value: 0.004732140339910984.
[I 2022-10-04 21:56:39,783] Trial 3 finished with value: 0.002529076999053359 and parameters: {'C': 0.12325972342278585}. Best is trial 3 with value: 0.002529076999053359.
[I 2022-10-04 21:56:39,945] Trial 4 finished with value: 0.002529076999053359 and parameters: {'C': 0.15006689332711087}. Best is trial 3 with value: 0.002529076999053359.
[I 2022-10-04 21:56:40,108

In [8]:
result_df

,TFe,MFe,FeO,Fe2O3,C,Zn,C_suggest
0,49.47,7.63,31.30,25.03,9.65,2.973,0.110968
1,50.00,7.48,31.34,25.96,9.38,2.096,0.076632


In [9]:
save_csv_1 = pd.DataFrame(result_df).to_csv('CC_OPT.csv')

In [25]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [ ]:
# print('best value is :\n', study_0.best_value)
# print('best condition is :\n',study_0.best_trial)

# list(study_0.best_trial.params.values())[0]

Step 4. Results - cross check
 - Is optimization working as we expected?
 - To show the carbon ratio controlled to meet the target dezincification ratio

In [10]:
TEST_df = pd.read_csv("CC_OPT_TEST.csv", encoding='cp949')

In [11]:
prediction_Z = predict_model (Model_Z, data = TEST_df/100)

In [12]:
save_csv_2 = pd.DataFrame(prediction_Z).to_csv('CC_OPT_result.csv')

In [21]:
c = pd.Series (np.linspace(8,11,10))

test_df_10 = pd.DataFrame({'TFe': 50, 'MFe':7.48 , 'FeO':31.34 , 'Fe2O3':25.96, 'C': c, 'Zn': 2.096} ) / 100

test_df_detail    = pd.concat([ test_df_10 ])

Detail_test_Z = predict_model (Model_Z, data = test_df_detail)

